In [1]:
import gymnasium as gym
from gymnasium import spaces, vector
import numpy as np
from scipy.stats import binom, nbinom, beta, poisson, gamma, norm, geom
import random
import ray
from ray import tune, air
from ray.rllib.algorithms.ppo import PPOConfig
from functools import partial
from itertools import combinations

2024-05-11 11:42:50,780	INFO util.py:159 -- Missing packages: ['ipywidgets']. Run `pip install -U ipywidgets`, then restart the notebook server for rich notebook output.
2024-05-11 11:42:51,020	INFO util.py:159 -- Missing packages: ['ipywidgets']. Run `pip install -U ipywidgets`, then restart the notebook server for rich notebook output.
2024-05-11 11:42:51,729	WARNING deprecation.py:50 -- DeprecationWarning: `DirectStepOptimizer` has been deprecated. This will raise an error in the future!


In [2]:
# !pip install pokerlib
# from pokerlib.enums import Rank, Suit, Hand
# from pokerlib import HandParser
# import random

# These are attempts for 2

In [2]:
import random
from collections import Counter
from itertools import combinations

import gymnasium as gym
from gym import spaces
from ray import tune

from gymnasium import spaces, vector
import numpy as np
from scipy.stats import binom, nbinom, beta, poisson, gamma, norm, geom
import ray
from ray import tune, air
from ray.rllib.algorithms.ppo import PPOConfig
from functools import partial
# Card and Deck definitions
class Card:
    def __init__(self, rank, suit):
        self.rank = rank
        self.suit = suit

    def __repr__(self):
        return f"{self.rank}{self.suit}"

class Deck:
    ranks = "23456789TJQKA"
    suits = "SHDC"

    def __init__(self):
        self.cards = [Card(rank, suit) for suit in self.suits for rank in self.ranks]
        random.shuffle(self.cards)

    def draw(self, count=1):
        return [self.cards.pop() for _ in range(min(count, len(self.cards)))]

# Game Environment
class CardGameEnv(gym.Env):
    metadata = {'render.modes': ['human']}
    hand_values = {
        'Straight Flush': 800, 'Four of a Kind': 420, 'Full House': 160,
        'Flush': 140, 'Straight': 120, 'Three of a Kind': 90,
        'Two Pair': 40, 'Pair': 20, 'High Card': 5
    }

    def __init__(self, config=None):
        super(CardGameEnv, self).__init__()
        self.deck = Deck()
        self.hand = self.deck.draw(8)
        self.discards = 3
        self.plays = 4
        self.total_score = 0
        self.action_space = spaces.Discrete(len(self.hand) + 1)  # Choose how many cards to play or discard
        self.observation_space = spaces.MultiDiscrete([13] * 8)  # Represent card ranks in hand

    # Other methods remain unchanged...

    def step(self, action):
        assert self.action_space.contains(action), f"Invalid Action: {action}"
        cards_to_play = random.sample(self.hand, action)
        self.play_cards(cards_to_play)
        reward = self.evaluate_hand(cards_to_play)
        # self.hand += self.deck.draw(action)
        self.total_score += reward
        done = self.plays == 0
        return self._get_obs(), reward, done, {}

    def reset(self):
        self.deck = Deck()
        self.hand = self.deck.draw(8)
        self.total_score = 0
        self.plays = 4
        self.discards = 3
        return self._get_obs()


    def _get_obs(self):
        # Initialize an array to hold the observations
        obs = np.zeros(8, dtype=np.int64)

        # Iterate over the hand of cards
        for i, card in enumerate(self.hand):
            # Get the index of the card's rank in the ranks list
            rank_index = self.deck.ranks.index(card.rank)
            # Set the observation value corresponding to the card's rank
            obs[i] = rank_index

        return obs


    def render(self, mode='human', close=False):
        print("Current hand:", self.hand)
        print("Score:", self.total_score)

    def play_cards(self, cards):
        for card in cards:
            self.hand.remove(card)

    def evaluate_hand(self, cards):
        if len(cards) < 5:
            return 0  # Less than 5 cards cannot form any of the hands listed (except for advanced rules)

        ranks = [card.rank for card in cards]
        suits = [card.suit for card in cards]
        rank_counts = Counter(ranks)
        suit_counts = Counter(suits)
        is_flush = max(suit_counts.values()) == 5
        sorted_ranks = sorted(ranks, key=lambda x: self.deck.ranks.index(x))
        rank_sequence = [self.deck.ranks.index(rank) for rank in sorted_ranks]
        is_straight = all(rank_sequence[i] + 1 == rank_sequence[i + 1] for i in range(len(rank_sequence) - 1))

        if is_flush and is_straight:
            return self.hand_values['Straight Flush']
        if max(rank_counts.values()) == 4:
            return self.hand_values['Four of a Kind']
        if sorted(rank_counts.values()) == [2, 3]:
            return self.hand_values['Full House']
        if is_flush:
            return self.hand_values['Flush']
        if is_straight:
            return self.hand_values['Straight']
        if max(rank_counts.values()) == 3:
            return self.hand_values['Three of a Kind']
        if len([rank for rank, count in rank_counts.items() if count == 2]) == 2:
            return self.hand_values['Two Pair']
        if len([rank for rank, count in rank_counts.items() if count == 2]) == 1:
            return self.hand_values['Pair']
        return self.hand_values['High Card']

# Training with RLlib using Config
config = (PPOConfig()
          .environment(CardGameEnv)
          .framework('torch')
          .training(gamma=1)
          .rollouts(num_rollout_workers=3)
)
stop = {"timesteps_total": 10}

tuner = tune.Tuner(
    "PPO",
    param_space=config.to_dict(),
    run_config=air.RunConfig(stop=stop),
)

res = tuner.fit()

ModuleNotFoundError: No module named 'gym'

# Good Env

In [14]:
import numpy as np
import gymnasium as gym
from gymnasium import spaces
from gymnasium.utils import seeding

suit_to_int = {'Hearts': 0, 'Diamonds': 1, 'Clubs': 2, 'Spades': 3}
val_to_int = {'2': 0, '3': 1, '4': 2, '5': 3, '6': 4, '7': 5, '8': 6, '9': 7, '10': 8, 'J': 9, 'Q': 10, 'K': 11, 'A': 12}
def card_to_int(card):
    """converts each cart to a unique integer

    Args:
        card (_type_): _description_

    Returns:
        _type_: _description_
    """
    return card[1] * 13 + card[0]

class CardGameEnv(gym.Env):
    def __init__(self, seed = None):
        self.action_space = spaces.MultiBinary(8)  # Binary array to represent discarding individual cards
        self.observation_space = spaces.Box(low=0, high=51, shape = [8] ,dtype=np.int16)
        self.suits = ['Hearts', 'Diamonds', 'Clubs', 'Spades']
        self.values = ['2', '3', '4', '5', '6', '7', '8', '9', '10', 'J', 'Q', 'K', 'A']
        self.deck =  [(value, suit) for value in range(0, 13) for suit in range(4)]
        self.seed()

    def seed(self, seed=None):
        self.np_random, seed = seeding.np_random(seed)
        return [seed]

    def reset(self, *, seed=None, options=None):
        self.hand = self.draw_hand(8)  # Number, Suit
        # print(self.hand)
        self.discards = 3
        self.plays = 4
        self.total_reward = 0
        return np.array([card_to_int(card) for card in self.hand]), {}

    def draw_hand(self, num_cards_to_draw):
        cards = []
        for i in range(num_cards_to_draw):
            card = random.choice(self.deck)
            # Remove the selected card from available deck
            self.deck.remove(card)
            cards.append(card)
        # print("Cards remaining:", len(self.deck))
        return cards

    # def step(self, action):
    #     print("Startinghand", self.hand)
    #     assert len(action) == 8

    #     discards = [idx for idx, val in enumerate(action) if val == 1]
    #     num_discards = len(discards)
    #     while len(discards) > 5:
    #         discards.pop()
    #         num_discards = len(discards)
    #     print("Discards", discards)
    #     assert 0 <= num_discards <= 5  # Agent can discard at most 5 cards and at least 0 cards
    #     assert num_discards < 8  # Agent must keep at least one card

    #     if num_discards > 0:  # Agent chose to discard specific cards
    #         num_new_cards = 8 - num_discards
    #         new_cards = self.draw_hand(num_new_cards)
    #         # Calculate remaining indices ensuring they are within the range of self.hand
    #         remaining_indices = [0,1,2,3,4,5,6,7]
    #         for i in discards:
    #             remaining_indices.remove(i)
    #         print(remaining_indices)
    #         print("hand", self.hand)
    #         self.hand = [self.hand[idx] for idx in remaining_indices] + new_cards

    #     done = self.plays == 0
    #     return np.array(self.hand), self.total_reward, done, {}

    def step(self, action):
        # print("Startinghand", self.hand)
        assert len(action) == 8

        discards = [idx for idx, val in enumerate(action) if val == 1]
        num_discards = len(discards)
        # Ensure that number of discards is at most 5
        while len(discards) > 5:
            discards.pop()
            num_discards = len(discards)
        # print("Discards", discards, len(discards))
        assert 0 <= num_discards <= 5  # Agent can discard at most 5 cards and at least 0 cards
        assert num_discards < 8  # Agent must keep at least one card

        if num_discards > 0:  # Agent chose to discard specific cards
            num_new_cards = num_discards
            new_cards = self.draw_hand(num_new_cards)
            # Calculate remaining indices ensuring they are within the range of self.hand
            remaining_indices = [i for i in range(8) if i not in discards]
            # print("Remaining Indices:", remaining_indices)
            # print("Hand before discarding:", self.hand)
            self.hand = [self.hand[idx] for idx in remaining_indices] + new_cards
            # print("Hand after discarding:", self.hand)

        self.discards -= 1
        if self.discards == 0 or self.plays == 0:
            done = True
        else:
            done = False
        
        reward = self.calculate_hand_value(self.hand)
        self.total_reward = reward

        return np.array([card_to_int(card) for card in self.hand]), self.total_reward, done, False, {}


    def calculate_hand_value(self, hand):
        # Generate all combinations of 5 cards
        combinations_5 = combinations(hand, 5)
        best_reward = 0
        for combo in combinations_5:
            card_values = sorted([card[0] for card in combo])
            card_suits = [card[1] for card in combo]
            counts = np.bincount(card_values)
            num_unique_cards = np.count_nonzero(counts)
            is_flush = len(set(card_suits)) == 1

            if num_unique_cards == 2:
                if 4 in counts:
                    reward = 420  # Four of a Kind
                else:
                    reward = 160  # Full House
            elif num_unique_cards == 3:
                if 3 in counts:
                    reward = 90  # Three of a Kind
                else:
                    reward = 40  # Two Pair
            elif num_unique_cards == 4:
                reward = 20  # Pair
            elif num_unique_cards == 5:
                is_straight = all(card_values[i] == card_values[i-1] + 1 for i in range(1, len(card_values)))
                if is_straight:
                    if is_flush:
                        reward = 800  # Straight Flush
                    else:
                        reward = 120  # Straight
                elif is_flush:
                    reward = 140  # Flush
                else:
                    reward = 5  # High Card
            
            if reward > best_reward:
                best_hand = combo
                best_reward = reward
        # print(best_hand)
        return best_reward

# Test the environment
env = CardGameEnv()
obs, _ = env.reset()
total_reward = 0
done = False
while not done:
    assert obs.shape 
    action = env.action_space.sample()  # Random action for testing
    obs, reward, done, _, _ = env.step(action)
    total_reward = reward
print("Reward:", total_reward)

# env.calculate_hand_value([(9, 2), (5, 2), (7, 1), (0, 2), (6, 2), (1, 0), (11, 0), (3, 0)])


Reward: 120


In [15]:
if ray.is_initialized():
  ray.shutdown()

runtime_env = {"py_modules": ["../.."]}
ray.init(runtime_env=runtime_env)

2024-05-11 11:48:20,319	INFO worker.py:1642 -- Started a local Ray instance.
2024-05-11 11:48:21,690	INFO packaging.py:518 -- Creating a file package for local directory '../..'.
2024-05-11 11:48:22,756	INFO packaging.py:346 -- Pushing file package 'gcs://_ray_pkg_077111a95853e78c.zip' (373.78MiB) to Ray cluster...
2024-05-11 11:48:23,144	INFO packaging.py:359 -- Successfully pushed file package 'gcs://_ray_pkg_077111a95853e78c.zip'.


Python version:,3.10.14
Ray version:,2.7.0


In [16]:
config = (PPOConfig()
          .environment(CardGameEnv)
          .framework('torch')
          .training(gamma=1,)
          .rollouts(num_rollout_workers=3)
)
stop = {"timesteps_total": 10}

tuner = tune.Tuner(
    "PPO",
    param_space=config.to_dict(),
    run_config=air.RunConfig(stop=stop),
)

res = tuner.fit()


2024-05-11 11:48:23,684	WARNING algorithm_config.py:2578 -- Setting `exploration_config={}` because you set `_enable_rl_module_api=True`. When RLModule API are enabled, exploration_config can not be set. If you want to implement custom exploration behaviour, please modify the `forward_exploration` method of the RLModule at hand. On configs that have a default exploration config, this must be done with `config.exploration_config={}`.
2024-05-11 11:48:28,395	ERROR tune_controller.py:1502 -- Trial task failed for trial PPO_CardGameEnv_0b72d_00000
Traceback (most recent call last):
  File "/opt/anaconda3/envs/RL/lib/python3.10/site-packages/ray/air/execution/_internal/event_manager.py", line 110, in resolve_future
    result = ray.get(future)
  File "/opt/anaconda3/envs/RL/lib/python3.10/site-packages/ray/_private/auto_init_hook.py", line 24, in auto_init_wrapper
    return fn(*args, **kwargs)
  File "/opt/anaconda3/envs/RL/lib/python3.10/site-packages/ray/_private/client_mode_hook.py", li

(pid=19611) DeprecationWarning: `DirectStepOptimizer` has been deprecated. This will raise an error in the future!
(PPO pid=19611) 2024-05-11 11:48:26,453	WARNING algorithm_config.py:2578 -- Setting `exploration_config={}` because you set `_enable_rl_module_api=True`. When RLModule API are enabled, exploration_config can not be set. If you want to implement custom exploration behaviour, please modify the `forward_exploration` method of the RLModule at hand. On configs that have a default exploration config, this must be done with `config.exploration_config={}`.
(PPO pid=19611) 2024-05-11 11:48:26,453	WARNING algorithm_config.py:672 -- Cannot create PPOConfig from given `config_dict`! Property __stdout_file__ not supported.
(RolloutWorker pid=19613) Exception raised in creation task: The actor died because of an error raised in its creation task, ray::RolloutWorker.__init__() (pid=19613, ip=127.0.0.1, actor_id=e84a4114b45c7a6f34b9f82101000000, repr=<ray.rllib.evaluation.rollout_worker.R